In [5]:
import pandas as pd
import numpy as np
import talib
import time
import json
import requests
from kiteconnect import KiteConnect
from apscheduler.schedulers.background import BackgroundScheduler
from datetime import datetime
import openpyxl
from twilio.rest import Client
import telegram

# Zerodha API Credentials
api_key = "your_api_key"
access_token = "your_access_token"
kite = KiteConnect(api_key=api_key)
kite.set_access_token(access_token)

# Telegram Notification Setup
telegram_bot_token = "your_telegram_bot_token"
telegram_chat_id = "your_chat_id"

def send_telegram_message(message):
    url = f"https://api.telegram.org/bot{telegram_bot_token}/sendMessage"
    data = {"chat_id": telegram_chat_id, "text": message}
    requests.post(url, data=data)

# Twilio WhatsApp Alerts Setup
twilio_account_sid = "your_twilio_sid"
twilio_auth_token = "your_twilio_auth_token"
twilio_whatsapp_number = "whatsapp:+14155238886"
user_whatsapp_number = "whatsapp:+your_number"
client = Client(twilio_account_sid, twilio_auth_token)

def send_whatsapp_message(message):
    client.messages.create(
        from_=twilio_whatsapp_number,
        body=message,
        to=user_whatsapp_number
    )

# Load NSE Stocks List
instruments = kite.instruments("NSE")
stocks = [stock["tradingsymbol"] for stock in instruments]

# Candlestick Patterns
CANDLE_PATTERNS = {
    "Bullish Engulfing": talib.CDLENGULFING,
    "Bearish Engulfing": talib.CDLENGULFING,
    "Hammer": talib.CDLHAMMER,
    "Shooting Star": talib.CDLSHOOTINGSTAR,
    "Morning Star": talib.CDLMORNINGSTAR,
    "Evening Star": talib.CDLEVENINGSTAR,
    "Doji": talib.CDLDOJI
}

def fetch_ohlc(stock, interval="5minute"):
    return kite.historical_data(kite.ltp(f"NSE:{stock}")[f"NSE:{stock}"]["instrument_token"],
                                from_date=datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                                to_date=datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                                interval=interval)

def check_trade_signals():
    for stock in stocks:
        try:
            data = fetch_ohlc(stock)
            df = pd.DataFrame(data)
            df['pattern_signal'] = 0
            for pattern, func in CANDLE_PATTERNS.items():
                df[pattern] = func(df['open'], df['high'], df['low'], df['close'])
                if df[pattern].iloc[-1] != 0:
                    df['pattern_signal'].iloc[-1] = df[pattern].iloc[-1]
                    signal = "LONG" if df[pattern].iloc[-1] > 0 else "SHORT"
                    message = (f"Stock: {stock}\n"
                               f"Pattern: {pattern}\n"
                               f"Signal: {signal}\n"
                               f"Entry Point: {df['close'].iloc[-1]}\n"
                               f"Stop Loss: {df['low'].iloc[-1] if signal == 'LONG' else df['high'].iloc[-1]}\n"
                               f"Take Profit Levels: TP1: {df['close'].iloc[-1] * 1.01}, TP2: {df['close'].iloc[-1] * 1.02}, TP3: {df['close'].iloc[-1] * 1.03}")
                    print(message)
                    send_telegram_message(message)
                    send_whatsapp_message(message)
        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")

# Trailing Stop Loss
def trailing_stop_loss(stock, trade_price, stop_loss, trail_amount=5):
    current_price = kite.ltp(f"NSE:{stock}")[f"NSE:{stock}"]["last_price"]
    if current_price >= trade_price + trail_amount:
        stop_loss += trail_amount / 2
    return stop_loss

# Auto Daily Trade Report
def generate_trade_report():
    trades = pd.read_csv("trades.csv")
    trades.to_excel("Daily_Trade_Report.xlsx", index=False)
    send_telegram_message("Daily trade report generated.")

# Scheduler Setup
scheduler = BackgroundScheduler()
scheduler.add_job(check_trade_signals, "interval", minutes=5)
scheduler.add_job(generate_trade_report, "cron", hour=23, minute=59)
scheduler.start()

print("🚀 Futures Trading Bot Started... Press Ctrl+C to stop.")

while True:
    try:
        time.sleep(60)
    except KeyboardInterrupt:
        print("Bot stopped.")
        scheduler.shutdown()
        break


🚀 Futures Trading Bot Started... Press Ctrl+C to stop.
Bot stopped.
